# TP2 : Classification de panneaux de circulation - GTSRB dataset

Le **jeu de données GTSRB** (German Traffic Sign Recognition Benchmark) est un jeu de données comportant **plus de 50,000 images** représentant **43 classes de panneaux de signalisation**. 

Ce dataset peut être téléchargé depuis ce [site](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign).

**NE PAS TELECHARGER LE DATASET**

<u>ATTENTION</u> : Le dataset est gros (626Mo)... Dans ce TP nous n'allons utilisé qu'une partie du dataset pour éviter des longs téléchargements : 30% des données d'entrainement, 10% des données de validation et 10% des données de test. Les données ont déjà été mélangées (shuffle), il n'est donc pas nécessaire de le faire.

---

### Dans ce TP vous allez :
- **Visualiser** les données d'entrainement
- Afficher **l'histogramme du nombre d'observation par classe** (ou population) pour le jeu d'entrainement et de test
- **Redimensionner toutes les images** dans un même format (30x30 pixels)
- Construire et comparer **différents réseaux de neurone** pour faire de la classification multi-classes pour ces 43 panneaux de signalisation.
- Utiliser des techniques de **data augmentation** afin d'améliorer les performances du modèle


## Step 1 - Import and Init

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model, clone_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
import random
import csv
from skimage import io

## Step 2 - Read and prepare the GTSRB dataset 

Etapes à suivre pour décompresser le dataset GTSRB sur le disque local de la VM Colab:
1. Ouvrir l'explorateur de fichiers et aller dans DocumentsCours/BUT/BUT2/IA/tp2
2. Récupérer le fichier GTSRB.zip et le copier sur votre Google drive
3. Exécuter les cellules suivantes

In [ ]:
# Montage du drive sur la VM
from google.colab import drive

drive.mount("/content/gdrive")
%cd /content/gdrive/MyDrive

# On decompresse le dataset depuis le drive vers le disque local /content/sample_data de la VM
!unzip -qq GTSRB.zip -d /content/sample_data

#### Let us define the path where the dataset is stored

In [ ]:
%cd /content
dataset_path = "sample_data"

#### Upload GTSRB compressed dataset (.zip)

In [1]:
#! [ ! -f GTSRB.zip ] && echo ".zip dosn't exist" && wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zV_jKW96OizJAPV_ZLbSFbAkaK4rD6CZ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zV_jKW96OizJAPV_ZLbSFbAkaK4rD6CZ" -O GTSRB.zip && rm -rf /tmp/cookies.txt

#### Unzip the dataset to the Colab machine

In [2]:
# if sample_data/RGB dosn't exist then exctract the dataset into sample_data/
#! [ ! -d /content/sample_data/train ] && echo "train doesn't exist" && unzip -qq GTSRB.zip -d /content/sample_data

#### Data are organized as follows :  

#### A function to download the train, validation and test sets 

In [ ]:
def load_dataset_from_folder(folder_path = 'train'):
    # data va contenir toutes les données d'entrainement
    x_train = []
    paths_list = []
    y_train = []
    classes = 43
    
    # Retrieving the images and their labels 
    for i in range(classes):
        # On parcours tous les sous-répertoires dans train
        # Il y a un répertoire par classe  
        path = os.path.join(folder_path, str(i))
        # Images est une liste qui contient de tous les fichiers du sous répertoire
        images = os.listdir(path)

        # On parcours toutes les images du sous-répertoire
        for a in images:
            try:
                # On ouvre l'image 
                image = Image.open(path + '/'+ a)
                # On redimensionne toutes les images en 30x30 pixels
                image = image.resize((30,30))
                # On enregistre chaque image redimensionnée dans un tableau numpy
                image = np.array(image)
        
                # Data contient toutes les images et leur label
                x_train.append(image) 
                # Data contient toutes les images et leur label
                y_train.append(i) 
            except:
                print("Error loading image")
    
    return np.asarray(x_train), np.asarray(y_train)

#### Load the dataset

In [ ]:
x_train, y_train  = load_dataset_from_folder(folder_path = dataset_path + '/train')
x_val, y_val      = load_dataset_from_folder(folder_path = dataset_path + '/val')
x_test, y_test    = load_dataset_from_folder(folder_path = dataset_path + '/test')

In [ ]:
# Affichage de la forme des données d'entrainement
print("Shape of train images is:", x_train.shape)
print("Shape of labels is:", y_train.shape)

### <font color="red">**Exo1**</font> : Afficher le nombre d'observations utilisées pour l'entrainement (train), la validation (val) et le test 

In [ ]:
# A COMPLETER
# ....

### Afficher quelques images du dataset (1 image par classe)

In [ ]:
# On crée un dictionnaire où la clef est le n° de la classe et la valeur le nom de la classe (description du panneau)
label_map = {
    0: '20_speed',
    1: '30_speed',
    2: '50_speed',
    3: '60_speed',
    4: '70_speed',
    5: '80_speed',
    6: '80_lifted',
    7: '100_speed',
    8: '120_speed',
    9: 'no_overtaking_general',
    10: 'no_overtaking_trucks',
    11: 'right_of_way_crossing',
    12: 'right_of_way_general',
    13: 'give_way',
    14: 'stop',
    15: 'no_way_general',
    16: 'no_way_trucks',
    17: 'no_way_one_way',
    18: 'attention_general',
    19: 'attention_left_turn',
    20: 'attention_right_turn',
    21: 'attention_curvy',
    22: 'attention_bumpers',
    23: 'attention_slippery',
    24: 'attention_bottleneck',
    25: 'attention_construction',
    26: 'attention_traffic_light',
    27: 'attention_pedestrian',
    28: 'attention_children',
    29: 'attention_bikes',
    30: 'attention_snowflake',
    31: 'attention_deer',
    32: 'lifted_general',
    33: 'turn_right',
    34: 'turn_left',
    35: 'turn_straight',
    36: 'turn_straight_right',
    37: 'turn_straight_left',
    38: 'turn_right_down',
    39: 'turn_left_down',
    40: 'turn_circle',
    41: 'lifted_no_overtaking_general',
    42: 'lifted_no_overtaking_trucks'
}

### <font color="red">**Exo2**</font> : Afficher 1 image du jeu d'entrainement pour chacune des 43 classes (celle que vous voulez) 
<u>Indications</u>:  
+ Faire une boucle de 0 à 42
+ Parcourir un à un les sous-réperoires (construire le chemin pour y accéder en utilisant le module os)
+ Récupérer les images du sous-répertoire
+ Ouvrir une des images et l'afficher
+ Mettre le nom du label en titre de l'image (utiliser pour cela le dictionnaire label_map défini plus haut)
+ Afficher plusieurs images par ligne (7 par exemple)

In [ ]:
classes = 43
plt.figure(figsize=(17, 30))   

# TO BE COMPLETED
# ....


### Comprendre le dataset GTSRB...
+ Ce dataset est fourni avec 3 fichiers CSV qui décrivent les données d'entrainement et de test
+ La 1ère ligne de ces fichiers contient les champs suivants : `Filename ; Width ; Height ; Roi.X1 ; Roi.Y1 ; Roi.X2 ; Roi.Y2 ; ClassId`

In [ ]:
df = pd.read_csv(f'{dataset_path}/Test.csv', header=0)   # lecture du contenu du fichier CSV sous forme d'un dataframe pandas 
display(df.head(20))   # On visualise les 20 premières lignes 

### <font color="red">**Exo3**</font> : 
+ Regarder attentivement la taille des images...
+ Est-ce que cela peut poser un problème pour un modèle de machine learning ?
+ Que faudrait-il faire à votre avis ?

In [ ]:
# Votre réponse :
# ...

### <font color="red">**Exo4**</font> :  Déterminer le nombre d'observations/echantillons par classe
---
+ Afficher l'histogramme du nombre d'échantillons d'entrainement et de test par classe (utiliser y_train et y_test)
+ Pour l'histogramme, essayer d'afficher le nom de chaque classe (par exemple '20_speed') avec un alignement vertical 
+ Afficher aussi le pourcentage d'échantillon pour chaque classe
+ Est-ce que ce jeu de données est équilibré (en termes de nombre d'échantillons par classe)?

<u>Indice</u> : utiliser le module matplotlib

In [ ]:
# TO BE COMPLETED
# ....


#### Affichage du pourcentage d'échantillon pour chaque classe

In [ ]:
# # TO BE COMPLETED
# ....


### Normalisation des données d'entrainement
Attention : changement de nom pour les données d'entrainement, de validation et de test (X en majuscule)

In [ ]:
X_train = x_train/255.0
X_val   = x_val/255.0
X_test  = x_test/255.0

In [ ]:
print("Shape of train images is:", X_train.shape)
print("Shape of labels is:", y_train.shape)

## Step 3 - Build a CNN model with Keras

### <font color="red">**Exo5**</font> : Construire un modèle CNN comportant les couches suivantes :   
+ 1 couche de **Convolution 2D** avec 32 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **MaxPooling 2D** de taille 2x2
+ 1 couche de **Convolution 2D** avec 64 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **MaxPooling 2D** de taille 2x2
+ 1 couche de **Convolution 2D** avec 128 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **MaxPooling 2D** de taille 2x2
---
+ 1 couche **Flatten**
+ 1 couche **Dense** de 128 neurones avec activation relu
+ 1 couche de **dropout** avec une probabilité de 0.5
+ 1 couche **Dense** de sortie de 43 neurones avec activation softmax

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(30, 30, 3), padding='same'))
# TO BE COMPLETED
# ....


In [ ]:
model.summary()

### <font color="red">**Exo6**</font> : Retrouver par le calcul le nombre de paramètres entrainables de ce modèle

In [ ]:
# Votre réponse :
# ...

## Step 4 - Train the model

#### Model Compilation

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Model Training

In [ ]:
# hyper-parameters
epochs      = 15
batch_size  = 32

In [ ]:
history = model.fit(X_train, y_train, 
                     batch_size=512, 
                     epochs=epochs, 
                     validation_data=(X_val, y_val))

#### Save the trained model

In [ ]:
model.save('model1.h5')  # always save your weights after training or during training

### <font color="red">**Exo7**</font> : Display history of the loss and Accuracy according to the number of epoch

Display loss and accuracy VS. epoch

In [ ]:
# TO BE COMPLETED
# ....


## Step 5 - Evaluate the model on test dataset

In [ ]:
pred = np.argmax(model.predict(X_test), axis=-1)

# Accuracy with the test data
print(f"Model accuracy on test data: {accuracy_score(y_test, pred)*100:2f}")

### <font color="red">**Exo8**</font> : Afficher precision, recall et f1-score du modèle

In [ ]:
# TO BE COMPLETED
# ....


### <font color="red">**Exo9**</font> : Afficher la matrice de confusion
Indication : la matrice de confusion est grande (43 par 43). Vous pouvez vous limitez à 10 ou 20 classes

In [ ]:
# TO BE COMPLETED
# ....


## Let's build another CNN model with Keras

Nous allons construire un autre modèle CNN afin de tenter d'améliorer les performances... 
### <font color="red">**Exo10**</font> : Construire un modèle CNN comportant les couches suivantes :   

+ 1 couche de **Convolution 2D** avec 16 filtres de taille 3x3 et activation 'relu'
+ 1 couche de **Convolution 2D** avec 32 filtres de taille 3x3 et activation 'relu'
+ 1 couche de **MaxPooling 2D** de taille 2x2
+ 1 couche de **BatchNormalization**
+ 1 couche de **Convolution 2D** avec 64 filtres de taille 3x3 et activation 'relu'
+ 1 couche de **Convolution 2D** avec 128 filtres de taille 3x3 et activation 'relu'
+ 1 couche de **MaxPooling 2D** de taille 2x2
+ 1 couche de **BatchNormalization**
---
+ 1 couche **Flatten**
+ 1 couche **Dense** de 512 neurones avec activation relu
+ 1 couche de **BatchNormalization**
+ 1 couche de **dropout** avec une probabilité de 0.5
+ 1 couche **Dense** de sortie de 43 neurones avec activation softmax

In [ ]:
# Building the model
model = keras.models.Sequential([    
# TO BE COMPLETED
# ....
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(30,30,3)),  # 1ère couche
    # ...
    # ...
    keras.layers.Dense(43, activation='softmax')      # Dernière couche
])

In [ ]:
# !!!!! Noter l'augmentation du nombre de paramètres entrainables par rapport au model1...
model.summary()

### On clone le modèle

In [ ]:
model2 = clone_model(model)

In [ ]:
# Hyper-parameters
lr = 0.001
epochs = 15  # ou 30
batch_size=32
opt = Adam(learning_rate=lr, beta_1 = lr/(epochs * 0.5))

### Model Compilation

In [ ]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

### Train the model

In [ ]:
history2 = model2.fit(X_train, y_train, 
                      batch_size=batch_size, 
                      epochs=epochs, 
                      validation_data=(X_val, y_val))

### Sauvegarde du modèle entrainé

In [ ]:
model2.save('model2.h5')  # always save your weights after training or during training

### Display history of the loss and Accuracy according to the number of epoch

In [ ]:
# TO BE COMPLETED
# ....


### Evaluate the model on test dataset

In [ ]:
pred = np.argmax(model2.predict(X_test), axis=-1)
# Accuracy with the test data
print(f"Model accuracy on test data: {accuracy_score(y_test, pred)*100:2f}")

# Utilisation de techniques de data augmentation

### On clone un nouveau modèle

In [ ]:
model3 = clone_model(model)

### Model Compilation

In [ ]:
model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### <font color="red">**Exo11**</font> : Etudier les différentes techniques de data augmentation qui vont être utilisées lors de l'entrainement du modèle

In [ ]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

### Entrainement du modèle avec de la data augmentation

In [ ]:
history3 = model3.fit(aug.flow(X_train, y_train, batch_size=32), epochs=15, validation_data=(X_val, y_val))

### Sauvegarde du modèle entrainé

In [ ]:
model3.save('model3.h5')  # always save your weights after training or during training

### Display history of the loss and Accuracy according to the number of epoch

In [ ]:
# TO BE COMPLETED
# ....
pd.DataFrame(history3.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 3)
plt.show()

### Validate the model on test dataset

In [ ]:
pred = np.argmax(model3.predict(X_test), axis=-1)
print(f"Model accuracy on test data: {accuracy_score(y_test, pred)*100:2f}")

### Conclusion : est-ce que la data augmentation a permis une amélioration des performances pour ce dataset ? 